# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [1]:
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

How the game looks (without our agent):

In [2]:
env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _,_,_ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
env.close()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
100%|██████████| 10/10 [00:05<00:00,  1.68it/s]


## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}

However, we don't know $Q^*$. So, we use neural network as a approximators, we can simply create one and train it to resemble $Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.
Think about the activation function you use at the last layer, which should depends the type of action you want out.

In [3]:
class Model(nn.Module):
  def __init__(self, observation_size, action_size):
    super(Model, self).__init__()
    # initialise layers here
    self.lvl1 = nn.Linear(observation_size, 64)
    self.lvl2 = nn.Linear(64, 32)
    self.lvl3 = nn.Linear(32, action_size)

  def forward(self, state):
    # send x through the network
     #state=torch.from_numpy(state)
     state.unsqueeze(0)
     x = torch.relu(self.lvl1(state))
     x = torch.relu(self.lvl2(x))
     x = torch.softmax(self.lvl3(x), dim=-1)#i choosed this because the action space is discerete

     return x

  def select_action(self, state):
        action_values = self.forward(state)
        action = torch.argmin(action_values, dim=-1).item()
        return action


### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [4]:
from numpy.lib.function_base import iterable

import random
class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size=observation_size
        self.action_size = action_size
        self.criterion = nn.MSELoss()

        self.model = Model(observation_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.maxlen=200
        self.memory =deque(maxlen=self.maxlen)  # memory that stores N most new transitions
        # good place to store hyperparameters as attributes
        self.batch_size = 20
        self.gamma = 0.6
        self.learning_rate = 0.5
        self.epsilon=0.2


    def remember(self, state, action, reward, next_state, done):
        # add to memory
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.maxlen:
            self.memory.pop(0)

    def act(self, state):
        # return an action from the model
        if random.random() < self.epsilon:
            return random.randint(0, self.action_size - 1)
        else:
            state = torch.tensor(state, dtype=torch.float32)
            q_values = self.model(state)
            return torch.argmax(q_values).item()

    def replay(self, batch_size):
        # update model based on replay memory
        # you might want to make a self.train() helper method
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)

        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.int64)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.float32)

        current_qv = self.model(states).gather(1, actions.unsqueeze(1))
        next_qv = self.model(next_states).max(dim=1, keepdim=True)[0]
        target_qv = rewards + (1 - dones) * self.gamma * next_qv
        loss = self.criterion(current_qv, target_qv)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()


### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [5]:
def train(env, agent, episodes=1000, batch_size=64):  # train for many games
     for _ in tqdm(range(episodes)):
         state = env.reset()
         done = False
         while not done:
             # 1. make a move in game.
             action = agent.act(state)
             next_state, reward, done, _ = env.step(action)
             # 2. have the agent remember stuff.
             agent.remember(state, action, reward, next_state, done)
             # 3. update state
             state = next_state
             # 4. if we have enough experiences in out memory, learn from a batch with replay.
             if len(agent.memory) >= batch_size:
                 agent.replay(batch_size)
     env.close()

### Putting it together
---
We train an agent on the environment:


In [6]:
env = gym.make('CartPole-v1')  # , render_mode='human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
torch.save(agent.model.state_dict(), 'model.pth')

  0%|          | 1/1000 [00:00<01:43,  9.63it/s]<ipython-input-4-380812cffba3>:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  states = torch.tensor(states, dtype=torch.float32)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([20, 20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 1000/1000 [00:25<00:00, 38.90it/s]


## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/